## For testing some of the options

Adapted from https://pbpython.com/bokeh-bullet-waterfall.html

In [1]:
from bokeh.io import show, output_notebook
from bokeh.palettes import Blues4 as pal # PuBu4
from bokeh.plotting import figure
from bokeh.models import Label, ColumnDataSource, LabelSet
from bokeh.models.formatters import NumeralTickFormatter

import pandas as pd

output_notebook()

Loading BokehJS ...

## This is for Bullet Graphs

In [2]:
data= [("John Smith", 105, 120),
       ("Jane Jones", 99, 110),
       ("Fred Flintstone", 109, 125),
       ("Barney Rubble", 135, 123),
       ("Mr T", 45, 105)]

limits = [0, 20, 60, 100, 160]
labels = ["Poor", "OK", "Good", "Excellent"]
cats = [x[0] for x in data]
chart_title = "Sales Rep Performance"

In [3]:
p=figure(title=chart_title, plot_height=350, plot_width=800, y_range=cats)
p.x_range.range_padding = 0
p.grid.grid_line_color = None
p.xaxis[0].ticker.num_minor_ticks = 0

for left, right, color in zip(limits[:-1], limits[1:], pal[::-1]):
    p.hbar(y=cats, left=left, right=right, height=0.8, color=color)

perf = [x[1] for x in data]
p.hbar(y=cats, left=0, right=perf, height=0.3, color="black")

comp = [x[2]for x in data]
p.segment(x0=comp, y0=[(x, -0.5) for x in cats], x1=comp,
          y1=[(x, 0.5) for x in cats], color="lime", line_width=2)

for start, label in zip(limits[:-1], labels):
    p.add_layout(Label(x=start, y=0, text=label, text_font_size="10pt",
                       text_color='black', y_offset=5, x_offset=15))

show(p)

## Waterfall Charts

In [4]:
# Create the initial dataframe
index = ['sales','returns','credit fees','rebates','late charges','shipping']
data = {'amount': [350000,-30000,-7500,-25000,95000,-7000]}
df = pd.DataFrame(data=data,index=index)

In [5]:
# Determine the total net value by adding the start and all additional transactions
net = df['amount'].sum()

df['running_total'] = df['amount'].cumsum()
df['y_start'] = df['running_total'] - df['amount']

# Where do we want to place the label?
df['label_pos'] = df['running_total']

df_net = pd.DataFrame.from_records([(net, net, 0, net)],
                                   columns=['amount', 'running_total', 'y_start', 'label_pos'],
                                   index=["net"])
df = df.append(df_net)

In [6]:
df['color'] = 'gray'

df.loc[df.amount < 0, 'color'] = 'maroon'
df.loc[df.amount < 0, 'label_pos'] = df.label_pos - 10000

#df.loc[df.amount > 0, 'color'] = 'green'

df["bar_label"] = df["amount"].map('{:,.0f}'.format)

In [7]:
TOOLS = "box_zoom,reset,save"
source = ColumnDataSource(df)
p = figure(tools=TOOLS, x_range=list(df.index), y_range=(0, net+40000),
           plot_width=800, title = "Sales Waterfall")

p.segment(x0='index', y0='y_start', x1="index", y1='running_total',
          source=source, color="color", line_width=55)

labels = LabelSet(x='index', y='label_pos', text='bar_label',
                  text_font_size="8pt", level='glyph',
                  x_offset=-20, y_offset=0, source=source)
p.add_layout(labels)

show(p)